# SQL Agent - Real Estate Database

This notebook serves as a simple demonstration of the core capabilities of the SQL agent. It is inspired by the repository at [Advanced QA and RAG Series](https://github.com/Farzad-R/Advanced-QA-and-RAG-Series?tab=readme-ov-file#Q&A-and-RAG-with-SQL-and-TabularData).

In this example, I load a subset of the data, consisting of 1,000 properties, focusing on a few columns that represent the most basic features.

I then use a SQL agent from LangChain ([documentation](https://api.python.langchain.com/en/latest/agent_toolkits/langchain_community.agent_toolkits.sql.base.create_sql_agent.html)) to query the data. To ensure the accuracy of the results, I also verify the queries using Pandas.



In [1]:
import pandas as pd
from pyprojroot import here

In [2]:
df = pd.read_csv(here("data/for_upload/properties_information_basic_features.csv"))
print(df.shape)
print(df.columns.tolist())
display(df.head(3))

(1000, 19)
['zpid', 'city', 'state', 'homeStatus', 'streetAddress', 'zipcode', 'bedrooms', 'bathrooms', 'price', 'longitude', 'latitude', 'homeType', 'lotSize', 'zestimate', 'rentZestimate', 'dateSoldString', 'taxAssessedValue', 'taxAssessedYear', 'parcelId']


,zpid,city,state,homeStatus,streetAddress,zipcode,bedrooms,bathrooms,price,longitude,latitude,homeType,lotSize,zestimate,rentZestimate,dateSoldString,taxAssessedValue,taxAssessedYear,parcelId
0,63662682,Greer,SC,SOLD,1008 Carriage Park Cir,29650,4.0,3.0,415000,-82.27929,34.867980,SINGLE_FAMILY,9583.0,581300.0,3153,2021-09-28,402970.0,2023.0,0534410100500
1,5716098,Lexington,NC,SOLD,1309 Riverwood Dr,27292,3.0,3.0,375000,-80.29264,35.654854,SINGLE_FAMILY,80586.0,447100.0,1900,2022-09-02,262980.0,2023.0,06036L0000006000
2,29134937,Arlington,TX,SOLD,1418 Cardinal St,76010,4.0,2.0,0,-97.08692,32.724987,SINGLE_FAMILY,9147.0,240000.0,1802,2022-05-25,213321.0,2023.0,03243176


### **SQL**


Most SQL databases make it easy to load a CSV file in as a table (DuckDB, SQLite, etc.). Once you’ve done this you can use all of the chain and agent-creating techniques outlined in the SQL use case guide. Here’s a quick example of how we might do this with SQLite:

In [ ]:
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine
db_path = str(here("data")) + "/csv_xlsx_sqldb.db"
db_path = f"sqlite:///{db_path}"

engine = create_engine(db_path)
# df.to_sql("titanic", engine, index=False)
df.to_sql("properties", engine, index=False)

For multiple csv files, we can create a sql with multiple tables:
```
df1.to_sql("csv1_name", engine, index=False)
df2.to_sql("csv2_name", engine, index=False)
```

**Lets make a test Query**

In [22]:
db = SQLDatabase(engine=engine)
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM properties WHERE state = 'WA';")

sqlite
['properties', 'properties_information']


"[(49400095, 'Rainier', 'WA', 'OTHER', '802 Tipsoo Loop S', 98576, 3.0, 1.25, 469900, -122.68318, 46.89975, 'MANUFACTURED', 67082.0, 469900.0, 2114, '2000-11-15', 429900.0, 2023.0, '63550014200'), (23232081, 'Vancouver', 'WA', 'OTHER', '3517 Grant St', 98660, 2.0, 1.0, 446900, -122.67795, 45.64753, 'SINGLE_FAMILY', 5349.0, 446900.0, 1994, '1975-08-01', 438766.0, 2023.0, '004480000'), (23610252, 'Bellingham', 'WA', 'OTHER', '66 Marigold Dr', 98229, 3.0, 3.0, 916500, -122.321144, 48.714054, 'SINGLE_FAMILY', None, 916500.0, 2614, '1999-10-18', 905898.0, 2023.0, '3704083694650000'), (67024958, 'Kelso', 'WA', 'OTHER', '709 S 5th Ave', 98626, 2.0, 1.0, 283900, -122.91032, 46.139755, 'SINGLE_FAMILY', 5662.0, 283900.0, 1700, '2013-01-18', 248450.0, 2023.0, '22746'), (49258449, 'Spanaway', 'WA', 'RECENTLY_SOLD', '21818 42nd Avenue E', 98387, 5.0, 3.0, 535000, -122.37244, 47.05826, 'SINGLE_FAMILY', 10001.0, 553400.0, 2895, '2023-11-06', 468400.0, 2023.0, '5001141430'), (67013600, 'Longview', 'WA

**Equivalent in Pandas**

In [21]:
df[df['state'] == 'WA']

,zpid,city,state,homeStatus,streetAddress,zipcode,bedrooms,bathrooms,price,longitude,latitude,homeType,lotSize,zestimate,rentZestimate,dateSoldString,taxAssessedValue,taxAssessedYear,parcelId
4,49400095,Rainier,WA,OTHER,802 Tipsoo Loop S,98576,3.0,1.25,469900,-122.683180,46.899750,MANUFACTURED,67082.0,469900.0,2114,2000-11-15,429900.0,2023.0,63550014200
23,23232081,Vancouver,WA,OTHER,3517 Grant St,98660,2.0,1.00,446900,-122.677950,45.647530,SINGLE_FAMILY,5349.0,446900.0,1994,1975-08-01,438766.0,2023.0,004480000
80,23610252,Bellingham,WA,OTHER,66 Marigold Dr,98229,3.0,3.00,916500,-122.321144,48.714054,SINGLE_FAMILY,NaN,916500.0,2614,1999-10-18,905898.0,2023.0,3704083694650000
190,67024958,Kelso,WA,OTHER,709 S 5th Ave,98626,2.0,1.00,283900,-122.910320,46.139755,SINGLE_FAMILY,5662.0,283900.0,1700,2013-01-18,248450.0,2023.0,22746
200,49258449,Spanaway,WA,RECENTLY_SOLD,21818 42nd Avenue E,98387,5.0,3.00,535000,-122.372440,47.058260,SINGLE_FAMILY,10001.0,553400.0,2895,2023-11-06,468400.0,2023.0,5001141430
209,67013600,Longview,WA,FOR_SALE,1 Monterrey Place,98632,3.0,3.00,579900,-122.943190,46.154310,SINGLE_FAMILY,9600.0,562200.0,2829,2001-10-03,399150.0,2023.0,02206339
252,38611480,Everett,WA,SOLD,1427 100th Street SW UNIT 40,98204,3.0,2.00,185000,-122.253880,47.910120,MANUFACTURED,NaN,202000.0,632,2021-07-09,193700.0,2023.0,00960003604000
303,49324498,Puyallup,WA,OTHER,14715 105th Avenue Ct E,98374,3.0,1.75,489000,-122.287980,47.122265,SINGLE_FAMILY,10637.0,489000.0,2767,1986-10-03,403100.0,2023.0,7760300030
441,96678923,Bellevue,WA,OTHER,7011 172nd Ave SE,98006,5.0,4.25,3538300,-122.111320,47.539288,SINGLE_FAMILY,13696.0,3538300.0,9432,2016-04-01,2526000.0,2023.0,0715010490
553,102645854,Spanaway,WA,SOLD,1619 177th Street E,98387,3.0,3.00,485000,-122.407740,47.096222,SINGLE_FAMILY,4900.0,547100.0,2999,2021-08-20,472400.0,2023.0,5003660410


### **Create an agent to interact with the Database**

Load credentials. Here we're utilizing an OpenAI model through Azure.

In [6]:
import os
from dotenv import load_dotenv
import warnings
warnings.filterwarnings("ignore")
print("Environment variables are loaded:", load_dotenv())
print("test by reading a variable:", os.getenv("OPENAI_API_TYPE"))

Environment variables are loaded: True
test by reading a variable: azure


In [7]:
from langchain.chat_models import AzureChatOpenAI

model_name = os.getenv("gpt_deployment_name")
azure_openai_api_key = os.environ["OPENAI_API_KEY"]
azure_openai_endpoint = os.environ["OPENAI_API_BASE"]
llm = AzureChatOpenAI(
    openai_api_version=os.getenv("OPENAI_API_VERSION"),
    azure_deployment=model_name,
    model_name=model_name,
    temperature=0.0)

c:\Users\danielcarvalho.dcf\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(


Now, let's run a test by querying the agent for properties located in Florida and Washington State.

In [24]:
from langchain_community.agent_toolkits import create_sql_agent
agent_executor = create_sql_agent(llm, db=db, verbose=True)

In [29]:
agent_executor.invoke({"input": "List the total houses in washington state and florida. also list the average price?"})



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: ""properties, properties_informationI should query the schema of the properties and properties_information tables to see what columns I can use to answer the question.
Action: sql_db_schema
Action Input: properties, properties_information
CREATE TABLE properties (
	zpid BIGINT, 
	city TEXT, 
	state TEXT, 
	"homeStatus" TEXT, 
	"streetAddress" TEXT, 
	zipcode BIGINT, 
	bedrooms FLOAT, 
	bathrooms FLOAT, 
	price BIGINT, 
	longitude FLOAT, 
	latitude FLOAT, 
	"homeType" TEXT, 
	"lotSize" FLOAT, 
	zestimate FLOAT, 
	"rentZestimate" BIGINT, 
	"dateSoldString" TEXT, 
	"taxAssessedValue" FLOAT, 
	"taxAssessedYear" FLOAT, 
	"parcelId" TEXT
)

/*
3 rows from properties table:
zpid	city	state	homeStatus	streetAddress	zipcode	bedrooms	bathrooms	price	longitude	latitude	homeType	lotSize	zestimate	rentZestimate	dateSoldString	taxAssessedValue	taxAssessedYear	parcelId
63662682	Greer	SC	SOLD	1008 Carriage Park Cir	2

{'input': 'List the total houses in washington state and florida. also list the average price?',
 'output': 'There are 31 houses in Washington state and 124 houses in Florida, with an average price of $717,996.77 and $557,047.46, respectively.'}

in the output we can look at the agent Query:

SELECT

state,

COUNT(*) as total_houses,

AVG(price) as avg_price

FROM properties

WHERE state IN ('WA', 'FL')
GROUP BY state

Let's check the answer with Pandas

In [32]:
filtered_df = df[df['state'].isin(['WA', 'FL'])]

# Group by 'state' and calculate the count and average price
result = filtered_df.groupby('state')['price'].agg(['count', 'mean']).reset_index()

# Rename the columns for clarity
result.columns = ['State', 'Property Count', 'Average Price']

# Display the result
result

,State,Property Count,Average Price
0,FL,124,557047.459677
1,WA,31,717996.774194


Let's introduce some typos to see if the agent/LLM can handle it. 

Let's also see if the agent understands that we now need the average considering both states at the same time.


In [30]:
agent_executor.invoke({"input": "what's the average price in washignto satete and floreida?"})



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: ""properties, properties_informationI should query the schema of the properties table to see if it has a price column.
Action: sql_db_schema
Action Input: properties
CREATE TABLE properties (
	zpid BIGINT, 
	city TEXT, 
	state TEXT, 
	"homeStatus" TEXT, 
	"streetAddress" TEXT, 
	zipcode BIGINT, 
	bedrooms FLOAT, 
	bathrooms FLOAT, 
	price BIGINT, 
	longitude FLOAT, 
	latitude FLOAT, 
	"homeType" TEXT, 
	"lotSize" FLOAT, 
	zestimate FLOAT, 
	"rentZestimate" BIGINT, 
	"dateSoldString" TEXT, 
	"taxAssessedValue" FLOAT, 
	"taxAssessedYear" FLOAT, 
	"parcelId" TEXT
)

/*
3 rows from properties table:
zpid	city	state	homeStatus	streetAddress	zipcode	bedrooms	bathrooms	price	longitude	latitude	homeType	lotSize	zestimate	rentZestimate	dateSoldString	taxAssessedValue	taxAssessedYear	parcelId
63662682	Greer	SC	SOLD	1008 Carriage Park Cir	29650	4.0	3.0	415000	-82.27929	34.86798	SINGLE_FAMILY	9583.0	581300.0	3153

{'input': "what's the average price in washignto satete and floreida?",
 'output': 'The average price in Washington state and Florida is $589,237.32.'}

In [33]:
filtered_df = df[df['state'].isin(['WA', 'FL'])]

# Calculate the overall average price for WA and FL combined
overall_avg_price = filtered_df['price'].mean()

# Display the result
print(f"The overall average price for properties in WA and FL is: ${overall_avg_price:.2f}")

The overall average price for properties in WA and FL is: $589237.32
